In [25]:
import os
import json
import time
import numpy as np
from google.cloud import storage

from venumpy import small_glwe as vp
from venumML.venum_tools import encrypt_array

from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split


In [26]:
ctx = vp.SecretContext()

In [27]:
# Step 1: Generate Sample Data
X, y = make_regression(n_samples=10, n_features=4, noise=0.1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, y_train.shape)

(8, 4) (8,)


In [28]:
X_train_ct = encrypt_array(X_train, ctx)
y_train_ct = encrypt_array(y_train, ctx)

In [29]:
# X_train_ct_json = [json.loads(ct.into_json()) for ct in X_train_ct]
X_train_ct_json = [[json.loads(ct.into_json()) for ct in row] for row in X_train_ct]

with open('X_train_ct.json', 'w') as json_file:
    # Use json.dump() to serialize the dictionary and write it to the file
    json.dump(X_train_ct_json, json_file, indent=4)  

In [30]:
y_train_ct_json = [json.loads(ct.into_json()) for ct in y_train_ct]

with open('y_train_ct.json', 'w') as json_file:
    # Use json.dump() to serialize the dictionary and write it to the file
    json.dump(y_train_ct_json, json_file, indent=4)  

In [31]:
def upload_to_gcp_bucket(file_paths, bucket_name, destination_blob_name=None):
    try:
        storage_client = storage.Client()
    except Exception as e:
        print(f"Failed to initialize GCP storage client: {str(e)}")
        return
    
    bucket = storage_client.bucket(bucket_name)
    
    for file_path in file_paths:
        try:
            # Use a unique blob name for each file when destination_blob_name is None
            blob_name = destination_blob_name if destination_blob_name else os.path.basename(file_path)
            blob = bucket.blob(blob_name)
            blob.upload_from_filename(file_path)
            print(f"File {file_path} uploaded to {bucket_name}/{blob_name} successfully!")
        except Exception as e:
            print(f"Error uploading {file_path}: {str(e)}")
            continue  # Continue with next file instead of stopping

if __name__ == "__main__":
    file_paths = ["./X_train_ct.json", "./y_train_ct.json"]
    bucket_name = "eml_training"
    upload_to_gcp_bucket(file_paths, bucket_name)

File ./X_train_ct.json uploaded to eml_training/X_train_ct.json successfully!
File ./y_train_ct.json uploaded to eml_training/y_train_ct.json successfully!
